In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import cobra
import cplex
import re
import os
from os.path import exists
import logging
from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
from escher import Builder
from optlang.symbolics import Zero, add
from modelseedpy import MSPackageManager, MSGapfill, FBAHelper, MSGrowthPhenotypes, MSModelUtil, MSATPCorrection
from modelseedpy.core.msgenome import MSGenome
from modelseedpy.core.rast_client import RastClient
from cobrakbase.core.kbasefba.newmodeltemplate_builder import NewModelTemplateBuilder
from modelseedpy.helpers import get_template
from sklearn.metrics import r2_score
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
print("Required modules loaded")

python version 3.7.6
cobrakbase 0.2.8
Required modules loaded


In [15]:
#Predicting essential genes in ARBank genomes on auxotrophic media
kbase_api = cobrakbase.KBaseAPI()
#List available models
modellist = kbase_api.ws_client.list_objects({
    "ids":[108937],
    "type":"KBaseFBA.FBAModel"
})
#Getting auxotrophic media
media = kbase_api.get_from_ws("Auxotrophy_media",108937)
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
#Iterating over models
data = {"Genome":[],"Growth":[],"Essential genes":[],"Essential reactions":[]}
for item in modellist:
    model = kbase_api.get_from_ws(item[1],item[6])
    pkgmgr = MSPackageManager.get_pkg_mgr(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
    solution=model.optimize()
    data["Genome"].append(item[1][0:-7])
    if solution.objective_value == 0:
        print(item[1]," failed to grow!")
        data["Growth"].append(0)
        data["Essential genes"].append("")
        data["Essential reactions"].append("")
    else:
        data["Growth"].append(solution.objective_value)
        genes = single_gene_deletion(model)
        genome = kbase_api.get_object(item[1][0:-7],item[6])
        trans_hash = {}
        transgenes = []
        for ftr in genome["features"]:
            if "db_xrefs" in ftr and len(ftr["db_xrefs"]) > 0:
                trans_hash[ftr["id"]] = ftr["db_xrefs"][0][1]
        for index, row in genes.iterrows():
            if row["growth"] < 0.01:
                if list(index)[0] in trans_hash:
                    transgenes.append(trans_hash[list(index)[0]])
        data["Essential genes"].append(",".join(transgenes))
        reactions = single_reaction_deletion(model)
        rxn_list = []
        for index, row in reactions.iterrows():
            if row["growth"] < 0.01:
                rxn_list.append(list(index)[0])
        data["Essential reactions"].append(",".join(rxn_list))
        df = pd.DataFrame(data)
        df.to_csv("Essentiality.csv")
HTML(df.to_html(render_links=True, escape=False))

AR_0457.GF.mdl  failed to grow!
AR_0440.GF.mdl  failed to grow!
AR_0239.GF.mdl  failed to grow!


,Genome,Growth,Essential genes,Essential reactions
0,AR_0607,2.380594,"fig|550.3129.peg.870,fig|550.3129.peg.4845,fig|550.3129.peg.2809,fig|550.3129.peg.277,fig|550.3129.peg.3121,fig|550.3129.peg.4469,fig|550.3129.peg.436,fig|550.3129.peg.4424,fig|550.3129.peg.1713,fig|550.3129.peg.2051,fig|550.3129.peg.1067,fig|550.3129.peg.2392,fig|550.3129.peg.4477,fig|550.3129.peg.995,fig|550.3129.peg.4935,fig|550.3129.peg.210,fig|550.3129.peg.209,fig|550.3129.peg.3133,fig|550.3129.peg.4949,fig|550.3129.peg.4673,fig|550.3129.peg.361,fig|550.3129.peg.977,fig|550.3129.peg.3944,fig|550.3129.peg.3059,fig|550.3129.peg.2495,fig|550.3129.peg.2562,fig|550.3129.peg.4846,fig|550.3129.peg.189,fig|550.3129.peg.3051,fig|550.3129.peg.656,fig|550.3129.peg.1686,fig|550.3129.peg.2773,fig|550.3129.peg.3431,fig|550.3129.peg.1387,fig|550.3129.peg.5126,fig|550.3129.peg.44,fig|550.3129.peg.597,fig|550.3129.peg.1970,fig|550.3129.peg.2380,fig|550.3129.peg.449,fig|550.3129.peg.2379,fig|550.3129.peg.1113,fig|550.3129.peg.3489,fig|550.3129.peg.208,fig|550.3129.peg.495,fig|550.3129.peg.598,fig|550.3129.peg.484,fig|550.3129.peg.4436,fig|550.3129.peg.2329,fig|550.3129.peg.3141,fig|550.3129.peg.2852,fig|550.3129.peg.890,fig|550.3129.peg.3127,fig|550.3129.peg.2496,fig|550.3129.peg.3084,fig|550.3129.peg.2830,fig|550.3129.peg.2377,fig|550.3129.peg.875,fig|550.3129.peg.2330,fig|550.3129.peg.3120,fig|550.3129.peg.1354,fig|550.3129.peg.1353,fig|550.3129.peg.339,fig|550.3129.peg.4137,fig|550.3129.peg.742,fig|550.3129.peg.272,fig|550.3129.peg.743,fig|550.3129.peg.2043,fig|550.3129.peg.45,fig|550.3129.peg.3066,fig|550.3129.peg.1116,fig|550.3129.peg.4444,fig|550.3129.peg.1642,fig|550.3129.peg.1678,fig|550.3129.peg.4485,fig|550.3129.peg.1673,fig|550.3129.peg.2824,fig|550.3129.peg.4423,fig|550.3129.peg.574,fig|550.3129.peg.1958,fig|550.3129.peg.3504,fig|550.3129.peg.3070,fig|550.3129.peg.3772,fig|550.3129.peg.4323,fig|550.3129.peg.3122,fig|550.3129.peg.3544,fig|550.3129.peg.2376,fig|550.3129.peg.3413,fig|550.3129.peg.4476,fig|550.3129.peg.2325,fig|550.3129.peg.1551,fig|550.3129.peg.2493,fig|550.3129.peg.2050,fig|550.3129.peg.206,fig|550.3129.peg.618,fig|550.3129.peg.4478,fig|550.3129.peg.2374,fig|550.3129.peg.3750,fig|550.3129.peg.777,fig|550.3129.peg.4451,fig|550.3129.peg.2444,fig|550.3129.peg.3078,fig|550.3129.peg.2624,fig|550.3129.peg.1559,fig|550.3129.peg.3131,fig|550.3129.peg.2625,fig|550.3129.peg.2385,fig|550.3129.peg.4472,fig|550.3129.peg.997,fig|550.3129.peg.3027,fig|550.3129.peg.346,fig|550.3129.peg.260,fig|550.3129.peg.3083,fig|550.3129.peg.1007,fig|550.3129.peg.435,fig|550.3129.peg.207,fig|550.3129.peg.1980,fig|550.3129.peg.3138,fig|550.3129.peg.344,fig|550.3129.peg.4471,fig|550.3129.peg.2831,fig|550.3129.peg.2626,fig|550.3129.peg.3432,fig|550.3129.peg.3783,fig|550.3129.peg.2375,fig|550.3129.peg.1967","rxn02175_c0,EX_cpd00009_e0,rxn05334_c0,EX_cpd00048_e0,rxn00029_c0,rxn02476_c0,EX_cpd00393_e0,rxn05340_c0,rxn05336_c0,rxn03164_c0,rxn05333_c0,rxn03908_c0,rxn13782_c0,EX_cpd00254_e0,EX_cpd00156_e0,rxn03638_c0,rxn03893_c0,rxn00710_c0,rxn00800_c0,rxn01513_c0,rxn01466_c0,rxn06848_c0,rxn05462_c0,rxn03137_c0,rxn00686_c0,rxn01643_c0,rxn02898_c0,rxn08336_c0,rxn05255_c0,rxn02008_c0,rxn11946_c0,rxn02341_c0,EX_cpd00322_e0,rxn05468_c0,rxn01538_c0,rxn00117_c0,rxn05463_c0,rxn00599_c0,rxn01997_c0,rxn02003_c0,EX_cpd00149_e0,rxn00910_c0,rxn04308_c0,rxn09106_c0,rxn01520_c0,rxn09693_c0,rxn05344_c0,rxn00693_c0,rxn08822_c0,rxn02774_c0,rxn03843_c0,rxn00122_c0,rxn09037_c0,rxn03909_c0,rxn00832_c0,rxn12510_c0,rxn01512_c0,rxn01485_c0,rxn09128_c0,rxn03511_c0,rxn00237_c0,rxn02056_c0,rxn05329_c0,rxn05328_c0,EX_cpd00104_e0,rxn06865_c0,rxn02937_c0,rxn02895_c0,rxn06991_c0,EX_cpd00063_e0,rxn08954_c0,rxn05324_c0,rxn00917_c0,rxn00555_c0,rxn05461_c0,rxn05332_c0,rxn03130_c0,rxn02405_c0,rxn10199_c0,rxn03150_c0,rxn02831_c0,rxn00100_c0,rxn03397_c0,rxn03910_c0,rxn01602_c0,rxn02404_c0,rxn08711_c0,rxn06023_c0,rxn03916_c0,rxn01213_c0,rxn05339_c0,rxn09225_c0,rxn03084_c0,rxn00060_c0,rxn08708_c0,rxn01740_c0,rxn03537_c0,

In [ ]:
#Pull E. coli Biolog dataset from KBase for use in simulation
kbpheno = kbase_api.get_object("ecoli_biolog", 93541)
biolog = MSGrowthPhenotypes.from_kbase_object(kbpheno,kbase_api)
mdlws = 119455
models = kbase_api.list_objects(mdlws, object_type="KBaseFBA.FBAModel",include_metadata=True)
#Loading cached data
#data = {}
f = open('BiologHash.json')
data = json.load(f)
gmm = kbase_api.get_from_ws("Carbon-Pyruvic-Acid","KBaseMedia")
template = kbase_api.get_from_ws("GramNegModelTemplateV4","NewKBaseModelTemplates")
for item in models:
    model = kbase_api.get_from_ws(item[1],mdlws)
    if item[1] not in data:
        data[item[1]] = {}
        model.objective = "bio1"
        msgapfill = MSGapfill(model,[template],[],{},{},[])
        gfresults = msgapfill.run_gapfilling(gmm,"bio1")
        model = msgapfill.integrate_gapfill_solution(gfresults)
        output = biolog.simulate_phenotypes(model,"bio1",True)
        for index, row in output["details"].iterrows():
            data[item[1]][row["Phenotype"]] = row["Simulated growth"]
        with open('BiologHash.json', 'w') as outfile:
            json.dump(data, outfile)

In [3]:
#Full essentiality analysis
kbase_api = cobrakbase.KBaseAPI()
deg_genome = MSGenome.from_fasta("DEG10.txt")
rast = RastClient()
res = rast.annotate_genome(deg_genome)
essential_functions = {}
for ftr in deg_genome.features:
    if "RAST" in ftr.ontology_terms:
        for func in ftr.ontology_terms["RAST"]:
            func = func.lower()
            func = re.sub("\s","",func)
            func = re.sub("[\d\-]+\.[\d\-]+\.[\d\-]+\.[\d\-]*","",func)
            func = re.sub("\#.*$","",func)
            func = re.sub("\(ec:*\)","",func)
            func = re.sub("[\(\)\[\],-]","",func)
            if func not in essential_functions:
                essential_functions[func] = []
            essential_functions[func].append(ftr.id)

In [ ]:
#Processed
processed = ["AR_0607","AR_0250","AR_0282","AR_0502","AR_0554","AR_0856","AR_0563","AR_0992","AR_0356","AR_0667","AR_0850","AR_0311","AR_0702","AR_0112","AR_0998","AR_0304","AR_0461","AR_0555","AR_0215","AR_0247","AR_0513","AR_0129","AR_0216","AR_0491","AR_0274","AR_0220","AR_0710","AR_0283","AR_0995","AR_0528","AR_1006","AR_0456","AR_0458","AR_0706","AR_0478","AR_0566","AR_0657","AR_0347","AR_0689","AR_0261","AR_0481","AR_0609","AR_0558","AR_0125","AR_0033","AR_0273","AR_0718","AR_0287","AR_0662","AR_0987","AR_0284","AR_0705","AR_0244","AR_0094","AR_0143","AR_0492","AR_0229","AR_0083","AR_0060","AR_0526","AR_1010","AR_0562","AR_1047","AR_0553","AR_0840","AR_0302","AR_0508","AR_0365","AR_0238","AR_0088","AR_0488","AR_1049","AR_0506","AR_0160","AR_0105","AR_0106","AR_0285","AR_0497","AR_0139","AR_1014","AR_0687","AR_0093","AR_0763","AR_0479","AR_0696","AR_1051","AR_0614","AR_0665","AR_0661","AR_0360","AR_0230","AR_0043","AR_0146","AR_0527","AR_1005","AR_0441","AR_1000","AR_0290","AR_0605","AR_0664","AR_0366","AR_0234","AR_0981","AR_0659","AR_0267","AR_0052","AR_0713","AR_0141","AR_0709","AR_0254","AR_0367","AR_0853","AR_0064","AR_0615","AR_0053","AR_0070","AR_0219","AR_0305","AR_0300","AR_0475","AR_0719","AR_0256","AR_0720","AR_0221","AR_0692","AR_0855","AR_0140","AR_0663","AR_0351","AR_0443","AR_0354","AR_1054","AR_0237","AR_0656","AR_0217","AR_0715","AR_0611","AR_0507","AR_0603","AR_0522","AR_0511","AR_0550","AR_0296","AR_0701","AR_0982","AR_0050","AR_0979","AR_0034","AR_0101","AR_0353","AR_0848","AR_0252","AR_0038","AR_0065","AR_0471","AR_0575","AR_0557","AR_0292","AR_0643","AR_0102","AR_0621","AR_1063","AR_0831","AR_0135","AR_1004","AR_1065","AR_0483","AR_0056","AR_0110","AR_0075","AR_0357","AR_0222","AR_0703","AR_0839","AR_0568","AR_0489","AR_0765","AR_0641","AR_0991","AR_0834","AR_0567","AR_0449","AR_0474","AR_0265","AR_0993","AR_0042","AR_0248","AR_0307","AR_0688","AR_0308","AR_0457","AR_0289","AR_0773","AR_0240","AR_0604","AR_0487","AR_0986","AR_0068"]
processed_hash = {}
for item in processed:
    processed_hash[item] = 1
#List available genomes and models
genlist = kbase_api.ws_client.list_objects({
    "ids":[108937],
    "type":"KBaseGenomes.Genome"
})
model_hash = {}
mdllist = kbase_api.ws_client.list_objects({
    "ids":[108937],
    "type":"KBaseFBA.FBAModel"
})
for mdl in mdllist:
    model_hash[mdl[1]] = mdl
protcomp_hash = {}
protcomplist = kbase_api.ws_client.list_objects({
    "ids":[108937],
    "type":"GenomeComparison.ProteomeComparison"
})
for protcomp in protcomplist:
    protcomp_hash[protcomp[1]] = protcomp
#Establishing dataframe structure
data = {"Genome":[],"Gene":[],"Reactions":[],"Blasthit":[],"Modelhit":[],"Annotationhit":[]}
#Getting auxotrophic media
media = kbase_api.get_from_ws("Auxotrophy_media",108937)
#Iterating over genomes
blacklist = ["phage","hypothetical","fig"]
for item in genlist:
    print(item[1])
    if item[1] in processed_hash:
        next
    genome = kbase_api.get_object(item[1],item[6])
    trans_hash = {}
    original_gene_trans = {}
    transgenes = []
    for ftr in genome["features"]:
        if "db_xrefs" in ftr and len(ftr["db_xrefs"]) > 0:
            original_gene_trans[ftr["db_xrefs"][0][1]] = ftr["id"]
            trans_hash[ftr["id"]] = ftr["db_xrefs"][0][1]
    gene_hash = {}
    #Checking for function-based essentiality
    for ftr in genome["features"]:
        for func in ftr["functions"]:
            origfunc = func
            func = func.lower()
            func = re.sub("\s","",func)
            func = re.sub("[\d\-]+\.[\d\-]+\.[\d\-]+\.[\d\-]*","",func)
            func = re.sub("\#.*$","",func)
            func = re.sub("\(ec:*\)","",func)
            func = re.sub("[\(\)\[\],-]","",func)
            if ftr["id"] in trans_hash and func in essential_functions:
                keep = 1
                for word in blacklist:
                    size = len(word)
                    
                    if func[0:size] == word:
                        keep = 0
                if keep == 1:
                    if trans_hash[ftr["id"]] not in gene_hash:
                        gene_hash[trans_hash[ftr["id"]]] = {}
                    gene_hash[trans_hash[ftr["id"]]]["function"] = origfunc+";"+";".join(essential_functions[func])
    #Checking for homology-based essentiality
    if item[1]+".DEG.hits" in protcomp_hash:
        protcomp = kbase_api.get_object(item[1]+".DEG.hits",item[6])
        count = 0
        for hits in protcomp["data1"]:
            geneid = protcomp["proteome1names"][count]
            if geneid in trans_hash:
                geneid = trans_hash[geneid]
                for hit in hits:
                    if geneid not in gene_hash:
                        gene_hash[geneid] = {}
                    if "blast" not in gene_hash[geneid]:
                        gene_hash[geneid]["blast"] = []
                    othergene = protcomp["proteome2names"][hit[0]][0:-5]
                    gene_hash[geneid]["blast"].append(othergene+":"+str(hit[0]))
            count += 1
    #Checking for model-based essentiality
    model = None
    if item[1]+".GF.mdl" in model_hash:
        model = kbase_api.get_from_ws(item[1]+".GF.mdl",item[6])
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
        solution=model.optimize()
        if solution.objective_value > 0:
            reactions = single_reaction_deletion(model)
            rxn_hash = {}
            for index, row in reactions.iterrows():
                if row["growth"] < 0.01:
                    rxn_hash[list(index)[0]] = 1
            genes = single_gene_deletion(model)
            for index, row in genes.iterrows():
                geneid = list(index)[0]
                if row["growth"] < 0.01:
                    if geneid in trans_hash:
                        geneobj = model.genes.get_by_id(geneid)
                        ess_rxns = []
                        for rxn in geneobj.reactions:
                            if rxn.id in rxn_hash:
                                ess_rxns.append(rxn.id)
                        geneid = trans_hash[geneid]
                        if geneid not in gene_hash:
                            gene_hash[geneid] = {}
                        gene_hash[geneid]["model"] = "Linked to essential reactions:"+";".join(ess_rxns)
    #Adding all essential genes to dataframe
    for gene in gene_hash:
        data["Genome"].append(item[1])
        data["Gene"].append(gene)
        mdlgene = original_gene_trans[gene]
        if model == None:
            data["Reactions"].append("No model")
        elif mdlgene in model.genes:
            rxn_list = []
            for rxn in model.genes.get_by_id(mdlgene).reactions:
                rxn_list.append(rxn.id)
            data["Reactions"].append(";".join(rxn_list))
        else:
            data["Reactions"].append("None")
        if "model" in gene_hash[gene]:
            data["Modelhit"].append(gene_hash[gene]["model"])
        else:
            data["Modelhit"].append("")
        if "blast" in gene_hash[gene]:
            data["Blasthit"].append(";".join(gene_hash[gene]["blast"]))
        else:
            data["Blasthit"].append("")
        if "function" in gene_hash[gene]:
            data["Annotationhit"].append(gene_hash[gene]["function"])
        else:
            data["Annotationhit"].append("")
    df = pd.DataFrame(data)
    df.to_csv("EssentialGenes2.csv")
HTML(df.to_html(render_links=True, escape=False))

AR_0607
AR_0250
AR_0282
AR_0502
AR_0554
AR_0856
AR_0563
AR_0992
AR_0356
AR_0667
AR_0850
AR_0311
AR_0702
AR_0112
AR_0998
AR_0304
AR_0461
AR_0555
AR_0215
AR_0247
AR_0513
AR_0129
AR_0216
AR_0491
AR_0274
AR_0220
AR_0710
AR_0283
AR_0995
AR_0528
AR_1006
AR_0456
AR_0458
AR_0706
AR_0478
AR_0566
AR_0657
AR_0347
AR_0689
AR_0261
AR_0481
AR_0609
AR_0558
AR_0125
AR_0033
AR_0273
AR_0718
AR_0287
AR_0662
AR_0987
AR_0284
AR_0705
AR_0244
AR_0094
AR_0143
AR_0492
AR_0229
AR_0083
AR_0060
AR_0526
AR_1010
AR_0562
AR_1047
AR_0553
AR_0840
AR_0302
AR_0508
AR_0365
AR_0238
AR_0088
AR_0488
AR_1049
AR_0506
AR_0160
AR_0105
AR_0106
AR_0285
AR_0497
AR_0139
AR_1014
AR_0687
AR_0093
AR_0763
AR_0479
AR_0696
AR_1051
AR_0614
AR_0665
AR_0661
AR_0360
AR_0230
AR_0043
AR_0146
AR_0527
AR_1005
AR_0441
AR_1000
AR_0290
AR_0605
AR_0664
AR_0366
AR_0234
AR_0981
AR_0659
AR_0267
AR_0052
AR_0713
AR_0141
AR_0709
AR_0254
AR_0367
AR_0853
AR_0064
AR_0615
AR_0053
AR_0070
AR_0219
AR_0305
AR_0300
AR_0475
AR_0719
AR_0256
AR_0720
AR_0221
AR_0692


In [2]:
#Loading CSV and analyzing to filter out false positives
df = pd.read_csv ('EssentialGenes2.csv')
function_hash = {}
for index, row in df.iterrows():
    if isinstance(row["Annotationhit"], str) and len(row["Annotationhit"]) > 0:
        array = row["Annotationhit"].split(";DEG")
        function = array[0]
        genes = array[1:]
        for i in range(0,len(genes)):
            genes[i] = "DEG"+genes[i]
        if function not in function_hash:
            function_hash[function] = {
                "total_blast_match":0,
                "blast_match_count":0,
                "deg_genes":genes,
                "genomes":{},
                "genome_genes":{},
                "reactions":{},
                "rxn_essentiality":{},
                "blast_hits":{}
            }
        function_hash[function]["genomes"][row["Genome"]] = 1
        function_hash[function]["genome_genes"][row["Gene"]] = 1
        if len(row["Reactions"]) > 0:
            array = row["Reactions"].split(";")
            for rxn in array:
                if rxn not in function_hash[function]["reactions"]:
                    function_hash[function]["reactions"][rxn] = 0
                function_hash[function]["reactions"][rxn] += 1
        matchcount = 0
        if isinstance(row["Blasthit"], str) and len(row["Blasthit"]) > 0:
            array = row["Blasthit"].split(";")
            for item in array:
                subarray = item.split(":")
                if subarray[0] not in function_hash[function]["blast_hits"]:
                    function_hash[function]["blast_hits"][subarray[0]] = subarray[1]
                elif subarray[1] > function_hash[function]["blast_hits"][subarray[0]]:
                    function_hash[function]["blast_hits"][subarray[0]] = subarray[1]
                if subarray[0] in genes:
                    matchcount += 1
        function_hash[function]["total_blast_match"] += matchcount
        if matchcount > function_hash[function]["blast_match_count"]:
            function_hash[function]["blast_match_count"] = matchcount
        if isinstance(row["Modelhit"], str) and len(row["Modelhit"]) > 0:
            array = row["Modelhit"].split(":")
            for i in range(1,len(array)):
                if array[i] not in function_hash[function]["rxn_essentiality"]:
                    function_hash[function]["rxn_essentiality"][array[i]] = 0
                function_hash[function]["rxn_essentiality"][array[i]] += 1
    elif isinstance(row["Modelhit"], str) and len(row["Modelhit"]) > 0:
        array = row["Modelhit"].split(":")
        for i in range(1,len(array)):
            if array[i] not in function_hash:
                function_hash[array[i]] = {
                    "total_blast_match":0,
                    "blast_match_count":0,
                    "deg_genes":[],
                    "genomes":{},
                    "genome_genes":{},
                    "reactions":{},
                    "rxn_essentiality":{},
                    "blast_hits":{}
                }
            for j in range(1,len(array)):
                function_hash[array[i]]["rxn_essentiality"][array[j]] = 1
            function_hash[array[i]]["genomes"][row["Genome"]] = 1
            function_hash[array[i]]["genome_genes"][row["Gene"]] = 1
            if len(row["Reactions"]) > 0:
                rxnarray = row["Reactions"].split(";")
                for rxn in rxnarray:
                    if rxn not in function_hash[array[i]]["reactions"]:
                        function_hash[array[i]]["reactions"][rxn] = 0
                    function_hash[array[i]]["reactions"][rxn] += 1
            matchcount = 0
            if isinstance(row["Blasthit"], str) and len(row["Blasthit"]) > 0:
                blastarray = row["Blasthit"].split(";")
                for item in blastarray:
                    subarray = item.split(":")
                    if subarray[0] not in function_hash[function]["blast_hits"]:
                        function_hash[array[i]]["blast_hits"][subarray[0]] = subarray[1]
                    elif subarray[1] > function_hash[function]["blast_hits"][subarray[0]]:
                        function_hash[array[i]]["blast_hits"][subarray[0]] = subarray[1]

In [9]:
output_data = {
    "Function":[],
    "Blast count":[],
    "Blast matches":[],
    "Essential gene count":[],
    "Essential genes":[],
    "Genomes":[],
    "Genome count":[],
    "Reaction count":[],
    "Reactions":[],
    "Essential reaction count":[],
    "Essential reactions":[],
    "Gene count":[]
}
for function in function_hash.keys():
    output_data["Function"].append(function)
    data = function_hash[function]
    output_data["Blast count"].append(data["total_blast_match"])
    output_data["Blast matches"].append(data["blast_match_count"])
    output_data["Essential gene count"].append(len(data["deg_genes"]))
    output_data["Essential genes"].append(",".join(data["deg_genes"]))
    output_data["Genomes"].append(",".join(data["genomes"]))
    output_data["Genome count"].append(len(data["genomes"]))
    output_data["Reaction count"].append(len(data["reactions"]))
    output_data["Reactions"].append(",".join(data["reactions"].keys()))
    output_data["Essential reaction count"].append(len(data["rxn_essentiality"]))
    output_data["Essential reactions"].append(",".join(data["rxn_essentiality"].keys()))
    output_data["Gene count"].append(len(data["genome_genes"]))
df = pd.DataFrame(output_data)
df.to_csv("Consolidated2.csv")

In [10]:
#This cell is meant to run after the previous cell to print the function hash
df = pd.DataFrame(function_hash)
df = df.transpose()
df.to_csv("Consolidated.csv")

In [2]:
#OLD CODE FOR SIMULATING KO ON HOPE COLLEGE GENOMES
kbase_api = cobrakbase.KBaseAPI()
#List available models
modellist = kbase_api.ws_client.list_objects({
    "ids":[93541],
    "type":"KBaseFBA.FBAModel"
})
#Iterate over the models and run FBA
LBMedia = model = kbase_api.get_from_ws("ArgonneLBMedia","KBaseMedia")
MMMedia = model = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
results = {}
count = 0
for item in modellist:
    model = kbase_api.get_from_ws(item[1],item[6])
    results[item[1]] = {"complete":{"genes":None,"reactions":None},"LB":{"genes":None,"reactions":None},"MM":{"genes":None,"reactions":None}}
    FBAHelper.set_objective_from_target_reaction(model, "bio1")
    pkgmgr = MSPackageManager.get_pkg_mgr(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(None)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in complete media")
        model = MSBuilder.gapfill_model(model,"bio1",template,None)
    print("Computing essentiality for "+item[1]+" in complete media")
    results[item[1]]["complete"]["genes"] = single_gene_deletion(model)
    results[item[1]]["complete"]["reactions"] = single_reaction_deletion(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(LBMedia)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in LB media")
        model = MSBuilder.gapfill_model(model,"bio1",template,LBMedia)
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        pkgmgr.getpkg("KBaseMediaPkg").build_package(LBMedia)
    print("Computing essentiality for "+item[1]+" in LB media")
    results[item[1]]["LB"]["genes"] = single_gene_deletion(model)
    results[item[1]]["LB"]["reactions"] = single_reaction_deletion(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(MMMedia)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in MM media")
        model = MSBuilder.gapfill_model(model,"bio1",template,MMMedia)
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        pkgmgr.getpkg("KBaseMediaPkg").build_package(MMMedia)
    print("Computing essentiality for "+item[1]+" in MM media")
    results[item[1]]["MM"]["genes"] = single_gene_deletion(model)
    results[item[1]]["MM"]["reactions"] = single_reaction_deletion(model)
    count += 1
    if count > 2:
        break

        
#Compute gene essentiality histogram from PATRIC functions
#Compute reaction essentiality histrogram from reaction IDs
#Simulate diverse carbon sources gapfilling on each carbon source using a model already gapfilled for minimal media
#Model dashboard: ATP, gapfilling, reaction counts, gene counts, 
#KEGG pathway analysis


Computing essentiality for 562.61280.fbamodel in complete media
Gapfilling 562.61280.fbamodel in LB media
Computing essentiality for 562.61280.fbamodel in LB media
Gapfilling 562.61280.fbamodel in MM media
Computing essentiality for 562.61280.fbamodel in MM media
Computing essentiality for 562.55367.fbamodel in complete media
Gapfilling 562.55367.fbamodel in LB media
Computing essentiality for 562.55367.fbamodel in LB media
Gapfilling 562.55367.fbamodel in MM media
Computing essentiality for 562.55367.fbamodel in MM media
Computing essentiality for 562.55368.fbamodel in complete media
Gapfilling 562.55368.fbamodel in LB media
Computing essentiality for 562.55368.fbamodel in LB media
Gapfilling 562.55368.fbamodel in MM media
Computing essentiality for 562.55368.fbamodel in MM media


In [14]:
contigs = {}
with open('Assembly.json') as f:
    contigs = json.load(f)
df = pd.read_csv("deg_annotation_p.csv",";")
with open('DEG.gff',"w") as f:
    f.write("##gff-version 3\n")
    for index, row in df.iterrows():
        if row["ID"] in contigs["contigs"]:
            f.write(row["ID"]+"\tDEG_export\tCDS\t1\t"+str(contigs["contigs"][row["ID"]]["length"])+"\t.\t+\t0\tID="+row["ID"]+"; product="+row["product"]+"\n")
        else:
            print(row["ID"])

DEG10010015
DEG10010020
DEG10010023
DEG10010024
DEG10010027
DEG10010030
DEG10010031
DEG10010034
DEG10010037
DEG10010039
DEG10010040
DEG10010042
DEG10010043
DEG10010044
DEG10010045
DEG10010046
DEG10010048
DEG10010049
DEG10010051
DEG10010053
DEG10010054
DEG10010058
DEG10010059
DEG10010060
DEG10010061
DEG10010063
DEG10010064
DEG10010065
DEG10010071
DEG10010073
DEG10010076
DEG10010082
DEG10010086
DEG10010097
DEG10010099
DEG10010100
DEG10010115
DEG10010119
DEG10010124
DEG10010126
DEG10010138
DEG10010143
DEG10010160
DEG10010174
DEG10010175
DEG10010183
DEG10010184
DEG10010187
DEG10010195
DEG10010196
DEG10010202
DEG10010205
DEG10010206
DEG10010225
DEG10010226
DEG10010228
DEG10010233
DEG10010249
DEG10010256
DEG10010265
DEG10010267
DEG10010269
DEG10010270
DEG10010271
DEG10020006
DEG10020012
DEG10020015
DEG10020019
DEG10020021
DEG10020027
DEG10020029
DEG10020036
DEG10020043
DEG10020046
DEG10020049
DEG10020050
DEG10020074
DEG10020075
DEG10020077
DEG10020091
DEG10020100
DEG10020117
DEG10020125
DEG1

DEG10060193
DEG10060195
DEG10060197
DEG10060198
DEG10060199
DEG10060201
DEG10060207
DEG10060215
DEG10060223
DEG10060232
DEG10060234
DEG10060235
DEG10060238
DEG10060265
DEG10060275
DEG10060278
DEG10060291
DEG10060292
DEG10060295
DEG10060296
DEG10060297
DEG10060298
DEG10060309
DEG10060317
DEG10060320
DEG10060321
DEG10060324
DEG10060327
DEG10060333
DEG10060335
DEG10060338
DEG10060339
DEG10060344
DEG10060346
DEG10060347
DEG10060357
DEG10060360
DEG10060362
DEG10060364
DEG10060375
DEG10060376
DEG10070005
DEG10070008
DEG10070022
DEG10070023
DEG10070027
DEG10070031
DEG10070045
DEG10070052
DEG10070054
DEG10070062
DEG10070071
DEG10070073
DEG10070077
DEG10070079
DEG10070096
DEG10070098
DEG10070106
DEG10070112
DEG10070122
DEG10070136
DEG10070143
DEG10070150
DEG10070160
DEG10070169
DEG10070170
DEG10070172
DEG10070181
DEG10070185
DEG10070199
DEG10070208
DEG10070210
DEG10070218
DEG10070220
DEG10070221
DEG10070224
DEG10070230
DEG10070231
DEG10070235
DEG10070238
DEG10070240
DEG10080002
DEG10080003
DEG1

DEG10160336
DEG10160341
DEG10160343
DEG10160344
DEG10160345
DEG10160350
DEG10160353
DEG10170003
DEG10170016
DEG10170017
DEG10170019
DEG10170022
DEG10170030
DEG10170033
DEG10170034
DEG10170039
DEG10170040
DEG10170043
DEG10170046
DEG10170047
DEG10170057
DEG10170062
DEG10170064
DEG10170083
DEG10170087
DEG10170089
DEG10170092
DEG10170094
DEG10170095
DEG10170097
DEG10170100
DEG10170102
DEG10170109
DEG10170111
DEG10170116
DEG10170118
DEG10170119
DEG10170122
DEG10170124
DEG10170125
DEG10170141
DEG10170145
DEG10170148
DEG10170151
DEG10170164
DEG10170165
DEG10170168
DEG10170171
DEG10170180
DEG10170184
DEG10170186
DEG10170193
DEG10170197
DEG10170198
DEG10170203
DEG10170211
DEG10170212
DEG10170220
DEG10170221
DEG10170222
DEG10170225
DEG10170230
DEG10170236
DEG10170237
DEG10170238
DEG10170241
DEG10170244
DEG10170245
DEG10170267
DEG10170268
DEG10170269
DEG10170276
DEG10170285
DEG10170286
DEG10170289
DEG10170290
DEG10170294
DEG10170298
DEG10170305
DEG10170306
DEG10170307
DEG10170309
DEG10170310
DEG1

DEG10250286
DEG10250327
DEG10250341
DEG10250343
DEG10250370
DEG10250371
DEG10250380
DEG10250383
DEG10250384
DEG10250390
DEG10250416
DEG10250421
DEG10250446
DEG10250485
DEG10250497
DEG10250524
DEG10250537
DEG10250610
DEG10250611
DEG10250617
DEG10250672
DEG10250673
DEG10250678
DEG10250679
DEG10250690
DEG10250692
DEG10250698
DEG10250708
DEG10250735
DEG10250771
DEG10260021
DEG10260025
DEG10260051
DEG10260065
DEG10260095
DEG10260098
DEG10270030
DEG10270082
DEG10270083
DEG10270110
DEG10270111
DEG10270118
DEG10270128
DEG10270129
DEG10270130
DEG10270235
DEG10270304
DEG10270342
DEG10270488
DEG10270607
DEG10270637
DEG10280005
DEG10280006
DEG10280010
DEG10280013
DEG10280014
DEG10280016
DEG10280019
DEG10280022
DEG10280023
DEG10280028
DEG10280030
DEG10280031
DEG10280032
DEG10280039
DEG10280041
DEG10280042
DEG10280049
DEG10280052
DEG10280054
DEG10280068
DEG10280069
DEG10280086
DEG10280088
DEG10280093
DEG10280105
DEG10280106
DEG10280107
DEG10280111
DEG10280112
DEG10280116
DEG10280117
DEG10280121
DEG1

DEG10380057
DEG10380058
DEG10380104
DEG10380113
DEG10380175
DEG10380209
DEG10380217
DEG10390017
DEG10390029
DEG10390031
DEG10390036
DEG10390041
DEG10390046
DEG10390049
DEG10390051
DEG10390059
DEG10390061
DEG10390065
DEG10390075
DEG10390091
DEG10390093
DEG10390113
DEG10390116
DEG10390117
DEG10390120
DEG10390124
DEG10390130
DEG10390133
DEG10390137
DEG10390139
DEG10390155
DEG10390157
DEG10390158
DEG10390160
DEG10390168
DEG10390171
DEG10390177
DEG10390180
DEG10390188
DEG10390191
DEG10390194
DEG10390196
DEG10390198
DEG10390201
DEG10390202
DEG10390208
DEG10390209
DEG10390217
DEG10390226
DEG10390233
DEG10390234
DEG10390235
DEG10390237
DEG10390238
DEG10390240
DEG10390242
DEG10390244
DEG10390245
DEG10390246
DEG10390248
DEG10390250
DEG10390254
DEG10390255
DEG10390275
DEG10400003
DEG10400012
DEG10400016
DEG10400039
DEG10400043
DEG10400052
DEG10400053
DEG10400064
DEG10400068
DEG10400069
DEG10400081
DEG10400085
DEG10400086
DEG10400090
DEG10400091
DEG10400099
DEG10400109
DEG10400111
DEG10400112
DEG1

DEG10470199
DEG10470202
DEG10470210
DEG10470211
DEG10470226
DEG10470227
DEG10470228
DEG10470230
DEG10470232
DEG10470241
DEG10470270
DEG10470272
DEG10470277
DEG10470292
DEG10470302
DEG10470311
DEG10470317
DEG10470344
DEG10470345
DEG10470348
DEG10470354
DEG10470359
DEG10470361
DEG10470377
DEG10470378
DEG10470393
DEG10470413
DEG10470419
DEG10470420
DEG10470423
DEG10470434
DEG10470436
DEG10470440
DEG10470441
DEG10470462
DEG10470465
DEG10470481
DEG10470486
DEG10470494
DEG10470495
DEG10480006
DEG10480009
DEG10480030
DEG10480055
DEG10480072
DEG10480074
DEG10480076
DEG10480077
DEG10480080
DEG10480081
DEG10480090
DEG10480096
DEG10480105
DEG10480106
DEG10480121
DEG10480122
DEG10480127
DEG10480128
DEG10480129
DEG10480134
DEG10480138
DEG10480140
DEG10480149
DEG10480158
DEG10480159
DEG10480160
DEG10480167
DEG10480172
DEG10480177
DEG10480180
DEG10480185
DEG10480186
DEG10480187
DEG10480189
DEG10480193
DEG10480198
DEG10480199
DEG10480204
DEG10480207
DEG10480209
DEG10480212
DEG10480216
DEG10480217
DEG1

DEG10550501
DEG10550502
DEG10550505
DEG10550506
DEG10550507
DEG10550508
DEG10550509
DEG10550513
DEG10550514
DEG10550518
DEG10550524
DEG10550528
DEG10550533
DEG10550535
DEG10550551
DEG10550552
DEG10550554
DEG10550559
DEG10550560
DEG10550561
DEG10550567
DEG10550569
DEG10550571
DEG10550577
DEG10550579
DEG10550580
DEG10550589
DEG10550600
DEG10550604
DEG10550606
DEG10550609
DEG10550610
DEG10550613
DEG10550614
DEG10550616
DEG10550619
DEG10550620
DEG10550622
DEG10550623
DEG10550624
DEG10550625
DEG10550627
DEG10550628
DEG10550630
DEG10550633
DEG10550635
DEG10550636
DEG10550638
DEG10550640
DEG10550643
DEG10550645
DEG10550646
DEG10550647
DEG10550648
DEG10550649
DEG10550655
DEG10550659
DEG10550663
DEG10550676
DEG10550677
DEG10550678
DEG10550679
DEG10550680
DEG10550692
DEG10550693
DEG10550694
DEG10550703
DEG10550704
DEG10550705
DEG10550706
DEG10550714
DEG10550716
DEG10550717
DEG10550725
DEG10550726
DEG10550729
DEG10550731
DEG10550745
DEG10550751
DEG10560003
DEG10560008
DEG10560009
DEG10560012
DEG1

DEG10610005
DEG10610026
DEG10610047
DEG10610069
DEG10610070
DEG10610104
DEG10610145
DEG10610146
DEG10610147
DEG10610148
DEG10610149
DEG10610150
DEG10610154
DEG10610157
DEG10610158
DEG10610159
DEG10610160
DEG10610161
DEG10610162
DEG10610167
DEG10610196
DEG10610202
DEG10610205
DEG10610206
DEG10610209
DEG10610213
DEG10610214
DEG10610215
DEG10610216
DEG10610217
DEG10610218
DEG10610219
DEG10610220
DEG10610223
DEG10610226
DEG10610227
DEG10610231
DEG10610232
DEG10610235
DEG10610238
DEG10610243
DEG10610248
DEG10610260
DEG10610261
DEG10610262
DEG10610273
DEG10610275
DEG10610276
DEG10610277
DEG10610278
DEG10610279
DEG10610280
DEG10610281
DEG10610282
DEG10610284
DEG10610285
DEG10610286
DEG10610287
DEG10620002
DEG10620004
DEG10620005
DEG10620026
DEG10620047
DEG10620069
DEG10620070
DEG10620104
DEG10620145
DEG10620146
DEG10620147
DEG10620148
DEG10620149
DEG10620150
DEG10620157
DEG10620158
DEG10620159
DEG10620160
DEG10620161
DEG10620162
DEG10620167
DEG10620196
DEG10620204
DEG10620206
DEG10620210
DEG1